In [ ]:
import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from lightgbm import plot_importance
import joblib
from hnmchallenge.constant import *
from hnmchallenge.data_reader import DataReader
from hnmchallenge.evaluation.python_evaluation import map_at_k, recall_at_k
from hnmchallenge.feature_manager import FeatureManager
from hnmchallenge.datasets.last_week_last_week import LWLWDataset
from hnmchallenge.datasets.last_month_last_week_dataset import LMLWDataset
from hnmchallenge.datasets.last_month_last_day import LMLDDataset
from hnmchallenge.models.itemknn.itemknn import ItemKNN
import lightgbm as lgb


In [ ]:
dataset = LMLDDataset()
dr = DataReader()

In [ ]:
base_load_path = dataset._DATASET_PATH / "lgbm_models"

In [ ]:
VERSION = 0
#DATASET = f"cutf_200_ItemKNN_tw_True_rs_False_{VERSION}.feather"
DATASET = f"cutf_500_ItemKNN_tw_True_rs_False_{VERSION}.feather"
MODEL_NAME = f"lgbm_{DATASET}.pkl"

In [ ]:
model=joblib.load(base_load_path/MODEL_NAME)

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
plot_importance(model, ax=ax)
plt.show()

In [ ]:
model.best_iteration_

In [ ]:
import re
DATASET = f"cutf_200_ItemKNN_tw_True_rs_False_{VERSION}.feather"
features = pd.read_feather(dataset._DATASET_PATH / f"dataset_dfs/full/{DATASET}")
features = features.rename(columns=lambda x: re.sub("[^A-Za-z0-9_]+", "", x))

# Remember to use the bert_ntree_limit when taking the predictions

In [ ]:
features.head()

In [ ]:
features.columns

In [ ]:
cat = [
    "index_code_gbm",
    "product_group_name_gbm",
    "index_group_name_gbm",
    "graphical_appearance_no_gbm",
]

cat_index = [i for i, c in enumerate(features.columns) if c in cat]
print(cat_index)

for col in cat:
    features[col] = pd.Categorical(features[col])

In [ ]:
customer_article_df = features[[DEFAULT_USER_COL, DEFAULT_ITEM_COL]].copy()
X = features.drop([DEFAULT_USER_COL, DEFAULT_ITEM_COL], axis=1)

In [ ]:
y_pred = model.predict(X, num_iteration=model.best_iteration_, njobs=-1)

In [ ]:
customer_article_df["predicted_score"] = y_pred

In [ ]:
customer_article_df

In [ ]:
sorted_scores = customer_article_df.sort_values([DEFAULT_USER_COL, "predicted_score"], ascending=[True, False])

In [ ]:
sorted_scores.head(20)

In [ ]:
sorted_scores_index = sorted_scores.reset_index(drop=True)

In [ ]:
cutoff = sorted_scores_index.groupby(DEFAULT_USER_COL).size().values
i=0 
filter_indices = []
for cut in cutoff:
    filter_indices.extend(range(i, i+12))
    i=i+cut

In [ ]:
cutoff.mean()

In [ ]:
filter_indices

In [ ]:
final_df = sorted_scores_index.loc[filter_indices]

In [ ]:
final_df

In [ ]:
final_final_df = final_df.drop("predicted_score", axis=1)

In [ ]:
dataset.create_submission(final_final_df, sub_name="gbm_rp3b_500_200")